In [0]:
from pyspark.sql.functions import count, avg, round

# Caminho do Silver e destino da Gold
silver_path = "dbfs:/Volumes/nycproject/default/silver/nyc_taxi"
gold_path = "dbfs:/Volumes/nycproject/default/gold/nyc_taxi_summary"

# Leitura da Silver
df_silver = spark.read.format("delta").load(silver_path)

# Agregação
df_gold = df_silver.groupBy("pickup_year", "pickup_month", "pickup_hour") \
    .agg(
        count("*").alias("total_trips"),
        round(avg("trip_duration_min"), 2).alias("avg_duration_min"),
        round(avg("trip_distance"), 2).alias("avg_distance_miles"),
        round(avg("total_amount"), 2).alias("avg_total_fare")
    )

# Gravar Gold particionada por ano/mês
df_gold.write.format("delta").mode("overwrite").partitionBy("pickup_year", "pickup_month").save(gold_path)


In [0]:
# Caminhos
bronze_path = "dbfs:/Volumes/nycproject/default/bronze/nyc_taxi"
silver_path = "dbfs:/Volumes/nycproject/default/silver/nyc_taxi"
gold_path   = "dbfs:/Volumes/nycproject/default/gold/nyc_taxi_summary"

# Leitura das camadas
df_bronze = spark.read.format("delta").load(bronze_path)
df_silver = spark.read.format("delta").load(silver_path)
df_gold   = spark.read.format("delta").load(gold_path)

# Validação de contagens
print("📊 Total de registros")
print(f"Bronze: {df_bronze.count()}")
print(f"Silver: {df_silver.count()}")
print(f"Gold: {df_gold.count()}")

# Visualização da tabela Gold
display(df_gold)

# Tabela simples: número de corridas por hora do dia
df_gold.createOrReplaceTempView("vw_gold_summary")
spark.sql("""
    SELECT pickup_hour, SUM(total_trips) AS total_corridas
    FROM vw_gold_summary
    GROUP BY pickup_hour
    ORDER BY total_corridas DESC
""").display()
